In [2]:
#from poloniex.public import PublicAPI

In [69]:
import requests
import datetime
import pytz

import logging
logger = logging.getLogger(__name__)


class PublicAPIError(Exception):
    """Ошибка при обращении к публичному API"""

    def __init__(self, message, method, body=None):
        self.message = message
        self.method = method
        self.body = body
        super().__init__(self.message)

    def __str__(self):
        message = "{0}. Method: {1}".format(self.message, self.method)
        if self.body is not None:
            message += ". Response: {0})".format(self.body)
        return message


class PublicAPI:
    """Класс реализует работу с публичным API poloniex.com"""
    PUBLIC_API_URL = "https://poloniex.com/public?command="
    PERIODS = [300, 900, 1800, 7200, 14400, 86400]

    def execute(self, command):
        try:
            print(self.PUBLIC_API_URL + command)
            response = requests.get(self.PUBLIC_API_URL + command).json()
        except Exception as e:
            raise PublicAPIError(e, command) from e

        if 'error' in response:
            raise PublicAPIError(response["error"], command, body=response)
        return response

    def get_orderbook(self, currencyPair, depth):
        """
        Returns the order book for a given market
        :param str currencyPair: A string that defines the market, "USDT_BTC" for example. Use "all" for all markets.
        :param int depth: Default depth is 50. Max depth is 100.
        :return: orderbook
        """
        command = "returnOrderBook&currencyPair={0}&depth={1}".format(currencyPair, depth)
        response = self.execute(command)
        return response

    def get_chartdata(self, currencyPair, period, start, end):
        """
        Returns candlestick chart data.
        :param str currencyPair: A string that defines the market, "USDT_BTC" for example.
        :param int period: Candlestick period in seconds. Valid values are 300, 900, 1800, 7200, 14400, and 86400.
        :param int start: The start of the window in seconds since the unix epoch.
        :param int end: The end of the window in seconds since the unix epoch.
        """
        if period not in self.PERIODS:
            raise PublicAPIError("bad_period", "returnChartData", body=period)

        if isinstance(start, str):
            start = self.date_to_unix_ts_in_utc(start)

        if isinstance(end, str):
            end = self.date_to_unix_ts_in_utc(end)

        command = "returnChartData&currencyPair={0}&start={1}&end={2}&period={3}".format(currencyPair, start, end,
                                                                                         period)
        response = self.execute(command)
        return response

    def get_trade_history(self, currencyPair, start, end):
        """
        Returns the past 200 trades for a given market,
        or up to 1,000 trades between a range specified in UNIX timestamps
        by the "start" and "end" GET parameters
        :param str currencyPair: A string that defines the market, "USDT_BTC" for example. Use "all" for all markets.
        :param int start: The start of the window in seconds since the unix epoch.
        :param int end: The end of the window in seconds since the unix epoch.
        :return: list of trade operation
        """
        if isinstance(start, str):
            start = self.date_to_unix_ts_in_utc(start)

        if isinstance(end, str):
            end = self.date_to_unix_ts_in_utc(end)

        command = "returnTradeHistory&currencyPair={0}&start={1}&end={2}"

        pages = []
        done = False
        while not done:
            response = self.execute(command.format(currencyPair, start, end))
            response.sort(key=lambda x: x["date"], reverse=True)

            if len(response):
                print(len(response))
                pages += response
                end = self.date_to_unix_ts_in_utc(response[-1]["date"]) - 1
                if end <= start:
                    done = True
            else:
                done = True
        return pages

    def get_trade_history_batch(self, currency_pair, start, end):
        """
        Returns the past 200 trades for a given market,
        or up to 1,000 trades between a range specified in UNIX timestamps
        by the "start" and "end" GET parameters
        :param str currency_pair: A string that defines the market, "USDT_BTC" for example. Use "all" for all markets.
        :param int start: The start of the window in seconds since the unix epoch.
        :param int end: The end of the window in seconds since the unix epoch.
        :return: list of trade operation
        """
        if isinstance(start, str):
            print("start to timestamp")
            start = self.date_to_unix_ts_in_utc(start)

        if isinstance(end, str):
            print("end to timestamp")
            end = self.date_to_unix_ts_in_utc(end)


        command = "returnTradeHistory&currencyPair={0}&start={1}&end={2}"
        done = False
        
        print("{0} -> {1}".format(start, end))
        while not done:
            print("===================================================")
            print("Download data from {0} to {1}".format(start, end))
            response = self.execute(command.format(currency_pair, start, end))
            print("len response: {}".format(len(response)))

            if len(response):
                response.sort(key=lambda x: x["date"], reverse=False)
                
                print("first record: {}".format(response[0]))
                print("last record: {}".format(response[-1]))

                yield response
                
                print("new end: {}".format(response[-1]["date"]))
            
                end = self.date_to_unix_ts_in_utc(response[-1]["date"]) - 1
                print("new end ts: {}".format(end))
                if end <= start:
                    done = True
            else:
                print("response zero lenght")
                done = True

            print("done: {}".format(done))

    def get_tickers(self):
        """
        Retrieves summary information for each currency pair listed on the exchange
        :return: dict with tickers
        """
        command = "returnTicker"
        response = self.execute(command)
        return response

    def get_24hvolume(self):
        """
        Returns the 24-hour volume for all markets as well as totals for primary currencies.
        :return: dict with tickers volume
        """
        command = "return24hVolume"
        response = self.execute(command)
        return response

    def get_currencies(self):
        """
        Returns the 24-hour volume for all markets as well as totals for primary currencies.
        :return: dict with tickers volume
        """
        command = "returnCurrencies"
        response = self.execute(command)
        return response

    @staticmethod
    def date_to_unix_ts_in_utc(date):
        timezone = pytz.timezone("UTC")
        without_timezone = datetime.datetime.strptime(date, "%Y-%m-%d %H:%M:%S")
        with_timezone = timezone.localize(without_timezone)
        return int(with_timezone.timestamp())


In [70]:
logger.setLevel(logging.DEBUG)

In [71]:
api = PublicAPI()

In [60]:
def _convert(value):
    try:
        converted = int(value)
    except Exception:
        converted = 0
    return converted


def _make_db_data(ticker, trades):
    data = []

    for trade in trades:
        order_number = _convert(trade.get("orderNumber", 0))
        g_trade_id = _convert(trade.get("globalTradeID", 0))
        trade_id = _convert(trade.get("tradeID", 0))

        if order_number is not None:
            order_number

        record = [
            api.date_to_unix_ts_in_utc(trade.get("date")),
            ticker,
            trade.get("type", "No_info"),
            float(trade.get("rate", 0.)),
            float(trade.get("amount", 0.)),
            float(trade.get("total", 0.)),
            order_number,
            g_trade_id,
            trade_id
        ]
        data.append(record)
    return data

In [72]:
# download params
ticker = 'USDT_USDD'
start_tm = '2022-08-10 00:00:00'
end_tm = '2022-08-10 00:05:00'

depth = 300
last_update = '2022-08-09 00:00:00'



start_ts = api.date_to_unix_ts_in_utc(start_tm)
end_ts = api.date_to_unix_ts_in_utc(end_tm)


print("  end_tm {0} ->   end_ts {1}".format(end_tm, end_ts))
print("start_tm {0} -> start_ts {1}".format(start_tm, start_ts))

print("Depth: {0:>44}".format(end_ts-start_ts))

  end_tm 2022-08-10 00:05:00 ->   end_ts 1660089900
start_tm 2022-08-10 00:00:00 -> start_ts 1660089600
Depth:                                          300


In [73]:
trades = api.get_trade_history(ticker, start_ts, end_ts)

https://poloniex.com/public?command=returnTradeHistory&currencyPair=USDT_USDD&start=1660089600&end=1660089900
100
https://poloniex.com/public?command=returnTradeHistory&currencyPair=USDT_USDD&start=1660089600&end=1660206223
100
https://poloniex.com/public?command=returnTradeHistory&currencyPair=USDT_USDD&start=1660089600&end=1660206223
100
https://poloniex.com/public?command=returnTradeHistory&currencyPair=USDT_USDD&start=1660089600&end=1660206223
100
https://poloniex.com/public?command=returnTradeHistory&currencyPair=USDT_USDD&start=1660089600&end=1660206223
100
https://poloniex.com/public?command=returnTradeHistory&currencyPair=USDT_USDD&start=1660089600&end=1660206223
100
https://poloniex.com/public?command=returnTradeHistory&currencyPair=USDT_USDD&start=1660089600&end=1660206223
100
https://poloniex.com/public?command=returnTradeHistory&currencyPair=USDT_USDD&start=1660089600&end=1660206223
100
https://poloniex.com/public?command=returnTradeHistory&currencyPair=USDT_USDD&start=1660

KeyboardInterrupt: 

In [ ]:
len(trades)

In [62]:
last_updates = {}

In [63]:

        
api.date_to_unix_ts_in_utc("2022-08-11 08:18:07")

1660205887

In [64]:
data_length_total = 0

for batch in api.get_trade_history_batch(ticker, start_ts, end_ts):
    data = _make_db_data(ticker, batch)
    if len(data):
        #self.conn.cursor.executemany(self.PUT_DATA_QUERY, data)
        #data_length = self.conn.cursor.rowcount
        data_length = len(data)
        data_length_total += data_length
        print("{0}: {1} records was written into db".format(ticker, data_length))
    else:
        print("{0}: No data to write".format(ticker))
last_updates[ticker] = ts_end

1660089600 -> 1660089900
Download data from 1660089600 to 1660089900
len response: 100
first record: {'globalTradeID': '60256329', 'tradeID': '60256329', 'date': '2022-08-11 08:20:57', 'type': '1', 'rate': '0.99988', 'amount': '157.59248663', 'total': '157.6114', 'orderNumber': None}
last record: {'globalTradeID': '60256424', 'tradeID': '60256424', 'date': '2022-08-11 08:23:18', 'type': '1', 'rate': '0.9996', 'amount': '43.29157644', 'total': '43.3089', 'orderNumber': None}
USDT_USDD: 100 records was written into db
new end: 2022-08-11 08:23:18
new end ts: 1660206197
done: False
Download data from 1660089600 to 1660206197
len response: 100
first record: {'globalTradeID': '60256329', 'tradeID': '60256329', 'date': '2022-08-11 08:20:57', 'type': '1', 'rate': '0.99988', 'amount': '157.59248663', 'total': '157.6114', 'orderNumber': None}
last record: {'globalTradeID': '60256424', 'tradeID': '60256424', 'date': '2022-08-11 08:23:18', 'type': '1', 'rate': '0.9996', 'amount': '43.29157644', '

len response: 100
first record: {'globalTradeID': '60256329', 'tradeID': '60256329', 'date': '2022-08-11 08:20:57', 'type': '1', 'rate': '0.99988', 'amount': '157.59248663', 'total': '157.6114', 'orderNumber': None}
last record: {'globalTradeID': '60256425', 'tradeID': '60256425', 'date': '2022-08-11 08:23:20', 'type': '2', 'rate': '0.99952', 'amount': '57.24440949', 'total': '57.2719', 'orderNumber': None}
USDT_USDD: 100 records was written into db
new end: 2022-08-11 08:23:20
new end ts: 1660206199
done: False
Download data from 1660089600 to 1660206199
len response: 100
first record: {'globalTradeID': '60256329', 'tradeID': '60256329', 'date': '2022-08-11 08:20:57', 'type': '1', 'rate': '0.99988', 'amount': '157.59248663', 'total': '157.6114', 'orderNumber': None}
last record: {'globalTradeID': '60256425', 'tradeID': '60256425', 'date': '2022-08-11 08:23:20', 'type': '2', 'rate': '0.99952', 'amount': '57.24440949', 'total': '57.2719', 'orderNumber': None}
USDT_USDD: 100 records was 

len response: 100
first record: {'globalTradeID': '60256329', 'tradeID': '60256329', 'date': '2022-08-11 08:20:57', 'type': '1', 'rate': '0.99988', 'amount': '157.59248663', 'total': '157.6114', 'orderNumber': None}
last record: {'globalTradeID': '60256426', 'tradeID': '60256426', 'date': '2022-08-11 08:23:24', 'type': '2', 'rate': '0.9996', 'amount': '6.32946720', 'total': '6.332', 'orderNumber': None}
USDT_USDD: 100 records was written into db
new end: 2022-08-11 08:23:24
new end ts: 1660206203
done: False
Download data from 1660089600 to 1660206203
len response: 100
first record: {'globalTradeID': '60256329', 'tradeID': '60256329', 'date': '2022-08-11 08:20:57', 'type': '1', 'rate': '0.99988', 'amount': '157.59248663', 'total': '157.6114', 'orderNumber': None}
last record: {'globalTradeID': '60256426', 'tradeID': '60256426', 'date': '2022-08-11 08:23:24', 'type': '2', 'rate': '0.9996', 'amount': '6.32946720', 'total': '6.332', 'orderNumber': None}
USDT_USDD: 100 records was written 

KeyboardInterrupt: 

In [ ]:
    
    
    
    
    def update(self, ticker, ts=None):
        data_length_total = 0

        config = self.config_manager.get_config().get("trades", dict())

        if ts is None:
            ts = int(time.time())

        last_update = self.last_updates.get(ticker, 0)
        logger.debug("[{0}] Local last update: {1}".format(ticker, last_update))
        if not last_update:
            last_update = self._get_max_ts(ticker)
            logger.debug("[{0}] DB last update: {1}".format(ticker, last_update))


        ts_end = ts
        ts_start = max(last_update, ts_end - config.get("depth", self.DEPTH_DEFAULT))

        logger.debug("[{0}] download period: {1} - {2}. Depth: {3}".format(ticker, ts_start, ts_end, ts_end-ts_start) )
        try:
            

        except PublicAPIError as e:
            logger.error("{0}: Data request error: {1}".format(ticker, e))
        return data_length_total